In [1]:
import json
from pathlib import Path

In [6]:
import json
from pathlib import Path


def load_json(path):
    with open(path, "r") as f:
        return json.load(f)


def get_pauli_labels_from_l8_fock(data):
    """
    Try multiple possibilities for L8_Fock.json:
    1) Direct pauli_labels list
    2) Reconstruct from ablation variants via removed_label
    """
    if "pauli_labels" in data and isinstance(data["pauli_labels"], list):
        return data["pauli_labels"]

    variants = data.get("variants", [])
    if isinstance(variants, list) and variants:
        labels = []
        for v in variants:
            lbl = v.get("removed_label")
            if isinstance(lbl, str):
                labels.append(lbl)

        # Deduplicate while preserving order
        seen = set()
        deduped = []
        for lbl in labels:
            if lbl not in seen:
                seen.add(lbl)
                deduped.append(lbl)

        if deduped:
            return deduped

    raise ValueError(
        "Could not find Pauli labels in L8_Fock.json "
        "(neither 'pauli_labels' nor variants[*]['removed_label'])."
    )


def compare_paulis_with_ground_metrics(
    linear8_path,
    l8_fock_path,
    save_output=True,
    output_name="pauli_comparison_linear8_vs_L8_Fock_with_ground_metrics.json",
):
    linear8 = load_json(linear8_path)
    l8_fock = load_json(l8_fock_path)

    # --- Required from linear_8 ---
    linear_labels = linear8.get("pauli_labels")
    if not isinstance(linear_labels, list):
        raise ValueError("linear_8.json must contain a 'pauli_labels' list.")

    # Optional aligned arrays from linear_8
    expvals_ground = linear8.get("pauli_expvals_ground", None)
    energy_contrib_ground = linear8.get("pauli_energy_contrib_ground", None)

    n = len(linear_labels)

    if expvals_ground is not None and len(expvals_ground) != n:
        raise ValueError(
            f"'pauli_expvals_ground' length ({len(expvals_ground)}) "
            f"does not match 'pauli_labels' length ({n})."
        )

    if energy_contrib_ground is not None and len(energy_contrib_ground) != n:
        raise ValueError(
            f"'pauli_energy_contrib_ground' length ({len(energy_contrib_ground)}) "
            f"does not match 'pauli_labels' length ({n})."
        )

    # --- Labels from L8_Fock ---
    l8_labels = get_pauli_labels_from_l8_fock(l8_fock)
    l8_set = set(l8_labels)

    # --- Build rows ---
    only_in_linear8 = []
    in_both = []

    for i, label in enumerate(linear_labels):
        row = {
            "index_in_linear_8": i,
            "pauli_label": label,
            "pauli_expvals_ground": expvals_ground[i] if expvals_ground is not None else None,
            "pauli_energy_contrib_ground": (
                energy_contrib_ground[i] if energy_contrib_ground is not None else None
            ),
        }

        if label in l8_set:
            in_both.append(row)
        else:
            only_in_linear8.append(row)

    # --- Print summary ---
    print(f"linear_8 labels: {len(linear_labels)}")
    print(f"L8_Fock labels:  {len(l8_labels)}")
    print(f"In both: {len(in_both)}")
    print(f"Only in linear_8: {len(only_in_linear8)}")

    # --- Print in both ---
    print("\n" + "=" * 80)
    print("PAULIS IN BOTH (with ground metrics)")
    print("=" * 80)
    for row in in_both:
        print(
            f"{row['pauli_label']:<20} "
            f"expval={row['pauli_expvals_ground']} "
            f"energy_contrib={row['pauli_energy_contrib_ground']}"
        )

    # --- Print only in linear_8 ---
    print("\n" + "=" * 80)
    print("PAULIS IN linear_8 BUT NOT IN L8_Fock (with ground metrics)")
    print("=" * 80)
    for row in only_in_linear8:
        print(
            f"{row['pauli_label']:<20} "
            f"expval={row['pauli_expvals_ground']} "
            f"energy_contrib={row['pauli_energy_contrib_ground']}"
        )

    # --- Save ---
    if save_output:
        out_path = Path(linear8_path).with_name(output_name)
        out_obj = {
            "linear8_path": str(linear8_path),
            "l8_fock_path": str(l8_fock_path),
            "counts": {
                "linear8_total": len(linear_labels),
                "l8_fock_total_labels": len(l8_labels),
                "in_both": len(in_both),
                "only_in_linear8": len(only_in_linear8),
            },
            "in_both": in_both,
            "only_in_linear8": only_in_linear8,
        }
        with open(out_path, "w") as f:
            json.dump(out_obj, f, indent=2)
        print(f"\nSaved results to: {out_path}")

    return in_both, only_in_linear8


In [7]:
linear8_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\Wess-Zumino\Analyses\Model Checks\GroundstatePauliContributions\dirichlet\linear\N3\linear_8.json"
l8_fock_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\Wess-Zumino\Qiskit\SKQD\ML-Testing\ML-Testing2\linear\N3\L8_Fock.json"

in_both, only_in_linear8 = compare_paulis_with_ground_metrics(
        linear8_path,
        l8_fock_path,
        save_output=False,
        )

linear_8 labels: 194
L8_Fock labels:  57
In both: 57
Only in linear_8: 137

PAULIS IN BOTH (with ground metrics)
IIIIIIIIIIII         expval=1.000000000000001 energy_contrib=12.250000000000014
IIIIIIIIIIZI         expval=0.9974128208554678 energy_contrib=-0.5610447117312006
IIIIIIIIIIZZ         expval=0.9923364331620248 energy_contrib=-0.558189243653639
IIIIIIIIIZII         expval=0.9999949109653523 energy_contrib=-1.6874914122540319
IIIIIIIIIZIZ         expval=0.9948793795737214 energy_contrib=-0.5596196510102183
IIIIIIIIIZZI         expval=0.9974077553968611 energy_contrib=-0.5610418624107343
IIIIIIIIIZZZ         expval=0.9923314934782442 energy_contrib=0.558186465081512
IIIIIIIIZIII         expval=0.9082482904638636 energy_contrib=-0.4541241452319318
IIIIIIZIIIII         expval=0.9898464624210701 energy_contrib=-0.6186540390131688
IIIIIIZZIIII         expval=0.9898464624210701 energy_contrib=-0.6186540390131688
IIIIIZIIIIII         expval=0.9999216693170163 energy_contrib=-1.8748531